In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\LENOVO\Desktop\Sample.csv")

In [3]:
df

,#,Question,"Quality of the question (Low, Medium, High)",Expected Answer,Initial Answer from Base Model (Mistral 7B),Initial Answer from LLM (gpt-4o-mini),Finetuned Mistral Model 1 (250 Tokens),Finetuned Mistral Model 1 (100 Tokens),BLUE Score (Base Mistral 7B vs gpt-4o-mini vs Finetuned Mistral Model 1),Finetuned Mistral Model 2 (250 Tokens),Finetuned Mistral Model 2 (100 Tokens),BLUE Score (Base Mistral 7B vs gpt-4o-mini vs Finetuned Mistral Model 2)
0,1.0,Why do we need health insurance?,Low,As medical care advances and treatments increa...,Health insurance is important because it helps...,Health insurance is important for several reas...,Health insurance is important because it prote...,Health insurance is important because it prote...,fine_tuned: BLEU = 0.0316\nbase_model: BLEU = ...,We all need health insurance because we never ...,Health Insurance is a must for all because of ...,NaN
1,2.0,How do I get health insurance?,Low,Most people get health insurance through their...,Health insurance is a policy that helps you pa...,Getting health insurance typically involves a ...,You can get health insurance by applying for a...,You can get health insurance by applying for a...,fine_tuned: BLEU = 0.0596\nbase_model: BLEU = ...,"To get health insurance, you typically need to...",Getting health insurance involves several step...,NaN
2,3.0,What is health insurance?,Low,Health insurance is an insurance product that ...,Health insurance is a type of insurance that h...,Health insurance is a type of coverage that pa...,Health insurance is a type of insurance that c...,Health insurance is a type of insurance that c...,fine_tuned: BLEU = 0.0328\nbase_model: BLEU = ...,Health insurance is an insurance product that ...,Health insurance is an insurance product which...,NaN
3,4.0,Why should I buy health insurance?,Low,You should purchase health insurance so that y...,Health insurance is a type of insurance that h...,Buying health insurance is important for sever...,Choosing the right health insurance plan invol...,You should buy health insurance because it is ...,fine_tuned: BLEU = 0.0162\nbase_model: BLEU = ...,Health insurance is a safety net for unforesee...,Health insurance is important for your financi...,NaN
4,5.0,How will health insurance pay for my emergency...,High,Your health insurance will either pay your hos...,Health insurance is a type of insurance that h...,Health insurance typically covers emergency me...,Your health insurance plan will cover emergenc...,Your health insurance plan will cover emergenc...,fine_tuned: BLEU = 0.0286\nbase_model: BLEU = ...,Your health insurance policy will cover a sign...,Your health insurance plan will cover the hosp...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['#', 'Question', 'Quality of the question (Low, Medium, High)',
       'Expected Answer', 'Initial Answer from Base Model (Mistral 7B)',
       'Initial Answer from LLM (gpt-4o-mini) ',
       'Finetuned Mistral Model 1 (250 Tokens)',
       'Finetuned Mistral Model 1 (100 Tokens)',
       'BLUE Score  (Base Mistral 7B  vs  gpt-4o-mini  vs  Finetuned Mistral Model 1)',
       'Finetuned Mistral Model 2 (250 Tokens)',
       'Finetuned Mistral Model 2 (100 Tokens)',
       'BLUE Score  (Base Mistral 7B  vs  gpt-4o-mini  vs  Finetuned Mistral Model 2)'],
      dtype='object')

In [2]:
import pandas as pd
import re
from tqdm import tqdm
from openai import OpenAI
import time
import json

# Initialize the OpenAI client
client = OpenAI(api_key=".............")

# Load dataset
df = pd.read_csv(r"C:\Users\LENOVO\Desktop\Sample.csv")

# Clean and rename columns
df.columns = df.columns.str.strip()
df = df.rename(columns={
    'Question': 'question',
    'Expected Answer': 'expected_answer',
    'Initial Answer from Base Model (Mistral 7B)': 'model_1',
    'Initial Answer from LLM (gpt-4o-mini)': 'model_2',
    'Finetuned Mistral Model 1 (100 Tokens)': 'model_3',
    'Finetuned Mistral Model 2 (100 Tokens)': 'model_4'
})

# Limit to first 60 rows
df = df.head(60)

# Text sanitizer
def sanitize(text):
    return str(text).replace("\n", " ").replace("\"", "'").strip()

# Evaluation function with reason (for 4 models)
def get_best_model_with_reason(question, expected_answer, model_1, model_2, model_3, model_4):
    system_prompt = (
        "You are a domain expert specializing in health insurance content evaluation. Your task is to assess four AI-generated answers "
        "against a reference (expected) answer and determine which model's output is **most semantically aligned** with the expected response.\n\n"
    
        "Instructions:\n"
        "1. Focus strictly on **semantic similarity** — assess the factual alignment, intent, and meaning.\n"
        "2. Ignore differences in grammar, tone, phrasing, or formatting.\n"
        "3. Select the model that best captures the **core information and intent** of the expected answer.\n"
        "4. If multiple answers are close, choose the one that conveys the expected information more precisely.\n"
        "5. Your output must strictly follow the JSON format below. Do not include any additional text.\n\n"
    
        "Output Format:\n"
        '{\n  "best_model": "Model X",\n  "reason": "Concise explanation (10-20 words) of why this answer best matches the expected answer." \n}'
    )

    user_prompt = f"""
Expected Answer: {expected_answer}

Model 1: {model_1}

Model 2: {model_2}

Model 3: {model_3}

Model 4: {model_4}

Which model's answer has the closest meaning to the expected answer? Only one model must be selected.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            max_tokens=100
        )
        content = response.choices[0].message.content.strip()

        try:
            parsed = json.loads(content)
            best_model = parsed.get("best_model", "Unknown")
            reason = parsed.get("reason", "No reason provided.")
            return best_model, reason
        except json.JSONDecodeError:
            return f"Unknown: {content}", "Invalid format from model."
    except Exception as e:
        return f"ERROR: {str(e)}", "API error"

# Evaluate and store results
best_models = []
reasons = []
backup_path = "model_best_choices_backup_4models.csv"

for idx, row in tqdm(df.iterrows(), total=len(df)):
    best_model, reason = get_best_model_with_reason(
        sanitize(row['question']),
        sanitize(row['expected_answer']),
        sanitize(row['model_1']),
        sanitize(row['model_2']),
        sanitize(row['model_3']),
        sanitize(row['model_4'])
    )
    best_models.append(best_model)
    reasons.append(reason)
    print(f"[{idx}] Best: {best_model} | Reason: {reason}")

    time.sleep(1)

    # Periodic backup every 10 rows
    if idx % 10 == 0:
        df['Best Model'] = best_models + [""] * (len(df) - len(best_models))
        df['Reason'] = reasons + [""] * (len(df) - len(reasons))
        df.to_csv(backup_path, index=False)

# Final save
df['Best Model'] = best_models
df['Reason'] = reasons
output_path = "model_best_choices_60_with_reason_4models.csv"
df.to_csv(output_path, index=False)

print(f"\nEvaluation complete. Results saved to: {output_path}")

  0%|          | 0/60 [00:00<?, ?it/s]

[0] Best: Model 2 | Reason: It comprehensively covers financial protection, access to care, preventive services, and peace of mind.


  2%|▏         | 1/60 [00:02<02:22,  2.41s/it]

[1] Best: Model 2 | Reason: It accurately describes obtaining health insurance through employers, government programs, and individual purchases.


  3%|▎         | 2/60 [00:05<02:28,  2.56s/it]

...

[58] Best: Model 2 | Reason: It accurately captures the tax implications for individuals and businesses regarding disability premiums.


 98%|█████████▊| 59/60 [02:32<00:02,  2.32s/it]

[59] Best: Model 3 | Reason: It accurately addresses claim denial reasons, including suicide and misrepresentation, aligning closely with the expected answer.


100%|██████████| 60/60 [02:34<00:00,  2.58s/it]


Evaluation complete. Results saved to: model_best_choices_60_with_reason_4models.csv


In [4]:
import pandas as pd
import json
import time
from tqdm import tqdm
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key="...................")

# Load your dataset
df = pd.read_csv("C:/Users/LENOVO/Desktop/Sample.csv")

# Clean column names
df.columns = df.columns.str.strip()
df = df.rename(columns={
    'Question': 'question',
    'Expected Answer': 'expected_answer',
    'Initial Answer from Base Model (Mistral 7B)': 'model_1',
    'Initial Answer from LLM (gpt-4o-mini)': 'model_2',
    'Finetuned Mistral Model 1 (100 Tokens)': 'model_3',
    'Finetuned Mistral Model 2 (100 Tokens)': 'model_4',
})


df = df.head(60)

# Clean text
def sanitize(text):
    return str(text).replace("\n", " ").replace("\"", "'").strip()

# Core Evaluation Function
def evaluate_with_llm_as_judge(question, expected, m1, m2, m3, m4):
    system_prompt = (
        "You are a health insurance expert evaluating the semantic similarity between four model answers "
        "and an expected answer. Select the one that best captures the core meaning and facts. "
        "Ignore wording differences. Choose **only one** best model.\n\n"
        "Output JSON format only:\n"
        '{\n  "best_model": "Model X",\n  "reason": "10-20 word explanation why this model is best." \n}'
    )

    user_prompt = f"""
Question: {question}

Expected Answer: {expected}

Model 1: {m1}
Model 2: {m2}
Model 3: {m3}
Model 4: {m4}

Which model is semantically closest to the expected answer?
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            max_tokens=150
        )

        content = response.choices[0].message.content.strip()
        try:
            parsed = json.loads(content)
            return parsed.get("best_model", "Unknown"), parsed.get("reason", "No reason provided.")
        except json.JSONDecodeError:
            return f"Parse Error: {content}", "Invalid JSON from model"

    except Exception as e:
        return f"ERROR: {str(e)}", "API error"

# Evaluate each row
best_models = []
reasons = []
backup_path = "llm_eval_backup.csv"

for idx, row in tqdm(df.iterrows(), total=len(df)):
    best_model, reason = evaluate_with_llm_as_judge(
        sanitize(row['question']),
        sanitize(row['expected_answer']),
        sanitize(row['model_1']),
        sanitize(row['model_2']),
        sanitize(row['model_3']),
        sanitize(row['model_4']),
    )

    best_models.append(best_model)
    reasons.append(reason)
    print(f"[{idx}] Best: {best_model} | Reason: {reason}")

    time.sleep(1.5)  # avoid rate limiting

    # Periodic backup every 10 rows
    if idx % 10 == 0:
        df['Best Model'] = best_models + [""] * (len(df) - len(best_models))
        df['Reason'] = reasons + [""] * (len(df) - len(reasons))
        df.to_csv(backup_path, index=False)

# Final save
df['Best Model'] = best_models
df['Reason'] = reasons
output_path = "llm_eval_results_with_model_4.csv"
df.to_csv(output_path, index=False)

print(f"\n Evaluation complete! Results saved to: {output_path}")


  0%|          | 0/60 [00:00<?, ?it/s]

[0] Best: Model 2 | Reason: It comprehensively covers financial protection, access to care, preventive services, and peace of mind, aligning closely with the expected answer.


  2%|▏         | 1/60 [00:04<04:10,  4.25s/it]

[1] Best: Model 2 | Reason: It comprehensively covers obtaining health insurance through employers, government programs, and individual plans, aligning closely with the expected answer.


  3%|▎         | 2/60 [00:06<03:14,  3.36s/it]

...

[58] Best: Model 2 | Reason: It accurately explains the tax implications for individuals and businesses, aligning closely with the expected answer.


 98%|█████████▊| 59/60 [03:04<00:02,  2.92s/it]

[59] Best: Model 3 | Reason: It accurately lists reasons for denial and emphasizes the importance of accurate information, aligning closely with the expected answer.


100%|██████████| 60/60 [03:07<00:00,  3.12s/it]


 Evaluation complete! Results saved to: llm_eval_results_with_model_4.csv


In [5]:
# Summary of Best Model Distribution
summary = df['Best Model'].value_counts(normalize=True) * 100
summary = summary.round(2).to_dict()

print("\nFinal Summary of Best Model Selection (%):")
for model, percent in summary.items():
    print(f"{model}: {percent}%")



Final Summary of Best Model Selection (%):
Model 2: 53.33%
Model 3: 23.33%
Model 4: 18.33%
Model 1: 5.0%
